In [1]:
import sys
import os
os.chdir('..')

In [49]:
from pathlib import Path
import pandas as pd
from collections import defaultdict
import requests 
from bs4 import BeautifulSoup

DATA = Path('data/version-3/')

ModuleNotFoundError: No module named 'requests'

In [9]:
def clean_colnames(df):
    df.columns = [col.replace(' ', '-').lower() for col in df.columns]

## Vanilla Associations

#### a. Food-Disease

In [3]:
food_dis = pd.read_csv(DATA/'food-disease.tsv', sep='\t', encoding='utf-8')
food_dis.drop_duplicates(['food-id', 'disease-id', 'association', 'pmid'], inplace=True)
food_dis = food_dis.where((pd.notnull(food_dis)), None)
food_dis['association'] = food_dis['association'].apply(str.lower)
food_dis = food_dis.loc[:, ['food-id', 'disease-id', 'association', 'pmid']]
food_dis['pmid'] = food_dis['pmid'].apply(str)
food2dis = defaultdict(lambda: set(),
                       food_dis.groupby('food-id').agg(lambda s: set(s))['disease-id'].to_dict())
food_dis.head(1)

,food-id,disease-id,association,pmid
0,Plant ID:4045,MESH:D004487,negative,7398283


#### b. Food-Chemical

In [4]:
food_chem = pd.read_csv(DATA/'food-chemical.tsv', sep='\t', encoding='utf-8')
food2chem = defaultdict(lambda: set(),
                        food_chem.groupby('food-id').agg(lambda s: set(s))['pubchem-id'].to_dict())

food_chem.head(1)

,food-id,pubchem-id,content,reference
0,Plant ID:100170,5280378,Detected but not quantified,KNApSAcK


#### c. Gene-Disease

In [5]:
gene_dis = pd.read_csv(DATA/'gene-disease.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
dis2gene = defaultdict(lambda: set(),
                        gene_dis.groupby('disease-id').agg(lambda s: set(s))['gene-id'].to_dict())
gene2dis = defaultdict(lambda: set(),
                        gene_dis.groupby('gene-id').agg(lambda s: set(s))['disease-id'].to_dict())

gene_dis.head()

,disease-id,gene-id,source
0,MESH:C000591739,1588,ORPHANET
1,MESH:C000596385,26504,UNIPROT
2,MESH:C000598645,9499,CTD_human;ORPHANET;UNIPROT
3,MESH:C000600608,29940,ORPHANET
4,MESH:C000600608,113189,CTD_human;ORPHANET;UNIPROT


#### d. Chemical-Disease

In [6]:
chem_dis = pd.read_csv(DATA/'chemical-disease.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
chem_dis = chem_dis.groupby(['pubchem-id', 'disease-id']).agg(lambda s: sorted(s)[-1]).reset_index()
dis2chem = defaultdict(lambda: set(),
                       chem_dis.groupby('disease-id').agg(lambda s: set(s))['pubchem-id'].to_dict())
chem2dis = defaultdict(lambda: set(),
                       chem_dis.groupby('pubchem-id').agg(lambda s: set(s))['disease-id'].to_dict())

chem_dis.head(1)

,pubchem-id,disease-id,type
0,19,MESH:D007674,therapeutic


#### e. Chemical-Gene

In [7]:
chem_gene = pd.read_csv(DATA/'chemical-gene.tsv', sep='\t', encoding='utf-8')
chem2gene = defaultdict(lambda: set(),
                        chem_gene.groupby('pubchem-id').agg(lambda s: set(s))['gene-id'].to_dict())
gene2chem = defaultdict(lambda: set(),
                        chem_gene.groupby('gene-id').agg(lambda s: set(s))['pubchem-id'].to_dict())
chem_gene.head()

,pubchem-id,gene-id,interaction-actions,source
0,19,351,affects^binding|decreases^reaction,CTD
1,19,4313,decreases^activity|decreases^reaction|decrease...,CTD
2,51,3091,affects^binding|affects^cotreatment|increases^...,CTD
3,51,7428,affects^binding|affects^cotreatment|increases^...,CTD
4,51,8290,affects^cotreatment|decreases^methylation|incr...,CTD


## Lexicons

### A. Food 

In [13]:
# Read file
foodlex = pd.read_csv(DATA/'food-lexicon-automated.tsv', sep='\t', encoding='utf-8')
clean_colnames(foodlex)

del foodlex['foodb-group'], foodlex['foodb-subgroup']

# Replace NaN with None
foodlex = foodlex.where((pd.notnull(foodlex)), None)

# Rename columns to match db schema.
foodlex.rename(columns={
    'common-names': 'common_names',
    'food-name': 'display_name',
    'food-id': 'food_id',
    'food-category': 'food_category',
    'scientific-name': 'scientific_name',
    'tax-id':'tax_id',
}, inplace=True)

foodlex.head()

,common_names,food_id,food_category,display_name,scientific_name,tax_id
0,alexanders; horse parsley,Plant ID:40962,Miscellaneous,Alexanders,Smyrnium olusatrum,40962
1,None,Plant ID:942083,Miscellaneous,Scrophularia umbrosa,Scrophularia umbrosa,942083
2,dotted blazing star,Plant ID:344074,Miscellaneous,Dotted blazing star,Liatris punctata,344074
3,plymouth pear,Plant ID:761867,Miscellaneous,Plymouth pear,Pyrus cordata,761867
4,None,Plant ID:49166,Miscellaneous,Rhododendron kaempferi,Rhododendron kaempferi,49166


In [24]:
# Redundancy check
assert len((set(food_dis['food-id']) | set(food_chem['food-id'])) - set(foodlex['food_id'])) == 0,\
    'Food lexicon is not exhaustive!'

### B. Disease

In [26]:
dislex = pd.read_csv(DATA/'CTD_diseases_new.tsv', sep='\t', encoding='utf-8', error_bad_lines=False)

# Keep only selected columns.
dislex = dislex[['DiseaseID', 'DiseaseName', 'SlimMappings', 'Synonyms']]

# Replace nan with None
dislex = dislex.where((pd.notnull(dislex)), None)

# Rename to match schema
dislex.rename(columns={
    'DiseaseID': 'disease_id',
    'DiseaseName': 'disease_name',
    'SlimMappings': 'disease_category',
    'Synonyms': 'disease_synonyms'
}, inplace=True)

dislex.head()

b'Skipping line 5374: expected 9 fields, saw 10\n'


,disease_id,disease_name,disease_category,disease_synonyms
0,MESH:C538288,10p Deletion Syndrome (Partial),Congenital abnormality|Genetic disease (inborn...,"Chromosome 10, 10p- Partial|Chromosome 10, mon..."
1,MESH:C535484,13q deletion syndrome,Congenital abnormality|Genetic disease (inborn...,Chromosome 13q deletion|Chromosome 13q deletio...
2,MESH:C579849,15q24 Microdeletion,Congenital abnormality|Genetic disease (inborn...,15q24 Deletion|15q24 Microdeletion Syndrome|In...
3,MESH:C579850,16p11.2 Deletion Syndrome,Congenital abnormality|Genetic disease (inborn...,None
4,MESH:C567076,"17,20-Lyase Deficiency, Isolated",Congenital abnormality|Endocrine system diseas...,"17-Alpha-Hydroxylase-17,20-Lyase Deficiency, C..."


In [29]:
# Redundancy check
assert len((set(food_dis['disease-id']) | set(chem_dis['disease-id']) | set(gene_dis['disease-id'])) 
           - set(dislex['disease_id'])) == 0,\
    'Disease lexicon is not exhaustive!'

# Subset disease lexicon
dislex = dislex.set_index('disease_id').\
    loc[list(set(food_dis['disease-id']) | set(chem_dis['disease-id']) | set(gene_dis['disease-id']))].\
    reset_index()

### C. Chemical

In [40]:
chem = pd.read_csv(DATA/'chemical-lexicon.tsv', sep='\t', encoding='utf-8').drop_duplicates(['pubchem_id'])

# Replace nan with null values
chem = chem.where((pd.notnull(chem)), None)

# Rename columns
chem.rename(columns={'canonical_smiles': 'smiles',},inplace=True)

del chem['inchi'], chem['inchikey']

In [43]:
# Redundancy check
assert len((set(food_chem['pubchem-id']) | set(chem_dis['pubchem-id']) | set(chem_gene['pubchem-id'])) 
           - set(chem['pubchem_id'])) == 0,\
    'Chemical lexicon is not exhaustive!'

### D. Gene

In [46]:
genes = list(set(gene_dis['gene-id']) | set(chem_gene))

In [48]:
len(genes)

8869